In [2]:
import re

In [4]:
with open('structure_tree.txt', 'r') as f:
    lines = f.readlines()
    lines = [l.rstrip() for l in lines if len(l) > 0]
    
    current = {}
    structure_list = {}
    
    for l in lines:
        if l.startswith('          - '):
            level = 3
            text = l.strip()[2:]
        elif l.startswith('     - '):
            level = 2
            text = l.strip()[2:]
        elif l.startswith('     '):
            level = 1
            text = l.strip()
        else:
            level = 0
            text = l.strip()
            
        if len(text) == 0:
            continue
        
        try:
            res = re.findall('^(.+?)(:\s?(.*))?$', text)[0]
        except:
            print 'text ERROR:', text
            raise
            
        name, _, desc = res
        
        if '[BEST]' in desc:
            goodness = 'best'
        elif '[GOOD]' in desc:
            goodness = 'good'
        else:
            goodness = None
            
        
        current[level] = name
        
        try:
            fullname, abbreviation = re.findall('^(.*),\s?(.*?)$', name)[0]
        except:
            fullname = name
            abbreviation = ''
        
        parent = current[level-1] if level > 0 else None
        
        structure_list[name] = {'level': level, 
               'fullname': fullname, 
               'abbr': abbreviation, 
               'description': desc,
                'goodness': goodness,
              'parent': parent,
                'children': []}
        
        if parent is not None:
            structure_list[parent]['children'].append(name)
        

In [12]:
print 'Best structures:'

for name, s in structure_list.iteritems():
    if s['goodness'] == 'best':
        print s['abbr']
        
print '\n'
print 'Good structures:'
        
for name, s in structure_list.iteritems():
    if s['goodness'] == 'good':
        print s['abbr']

Best structures:
fr
Amb
Tz
Pn
RtTg
SNR
7n
AP
10N
VLL
4N
6N
SNC
7N
12N
pc


Good structures:
Sol
LC
VTA
IP
5N
